In [35]:
import sys
sys.path.insert(0, '../src')
sys.path.insert(0, 'src') #for running notebook from project root
import numpy as np
import lds_regression as lr
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import xarray as xr
from evaluate_all_datasets import *
%matplotlib inline

center_out_path = '/home/pmalonis/lfads_analysis/data/intermediate/center_out/rockstar.nc'
dataset = xr.open_dataset(center_out_path)
s = center_out_gen_counts(dataset)

d_latent = 20
d_obs = s[0].shape[1]
n_trials = len(s)
sim_bin = np.diff(dataset.time.values)[0]
sim_bin = np.around(sim_bin, 3)
fit_bin = 0.02

In [36]:
def report_values(r2, A, pca):
    print('One step prediction r^2: %f'%r2)
    print('Fitted transition matrix:')
    print(A)
    print('True matrix:')
    print(lr.transition_mat)
    print('\nFitted transition matrix eigenvalues:')
    print(np.linalg.eigvals(A))
    print('True eigenvalues:')
    print(np.linalg.eigvals(lr.transition_mat))
    subspace_corr = np.corrcoef(W.flatten(), pca.components_.T.flatten())[1,0]
    print('True and fitted subspace correlation:%f'%subspace_corr)

In [37]:
def get_performance_single_trial(smooth_std, fit_bin):
    smoothed, b = lr.smooth_spikes_unequal(s, sim_bin, fit_bin, smooth_std)
    train = smoothed[:int(n_trials/2)]
    A, pca = lr.fit_system(train, d_latent, fit_bin)
    test = smoothed[int(n_trials/2):]
    p,a = lr.one_step_diff_explained(test, A, pca, fit_bin)
    r2 = np.corrcoef(np.concatenate(p).flatten(), np.concatenate(a).flatten())[1,0]**2
    #r2 = r2_score(np.concatenate(a), np.concatenate(p))
    return r2, A, pca

#iterative search for best smoothing parameters and fit bin parameters

r2 = []
smooth_stds = np.arange(0.01, 0.1, 0.01)
fit_bins = np.arange(0.01, 0.1, 0.01)
for smooth_std in smooth_stds:
    smooth_std = np.around(smooth_std, 2)
    r2.append(get_performance_single_trial(smooth_std, fit_bin)[0])
    
smooth_std = smooth_stds[np.argmax(r2)]
r2 = []
fit_bins = np.arange(.01, .1, .01)
for fit_bin in fit_bins:
    fit_bin = np.around(fit_bin, 2)
    r2.append(get_performance_single_trial(smooth_std, fit_bin)[0])
    
fit_bin = fit_bins[np.argmax(r2)]

In [38]:
r2, A, pca = get_performance_single_trial(smooth_std, fit_bin)
report_values(r2, A, pca)

One step prediction r^2: 0.000415
Fitted transition matrix:
[[-9.16735824e-01  6.27505020e-01 -9.25504937e-01  4.00461040e-02
  -6.00834991e-01 -1.91827496e-01 -1.45148234e+00  1.15950316e+00
   8.29859338e-01  6.61959831e-01 -6.10054398e-01 -1.09724186e+00
  -1.75285321e-01  1.82265989e-01 -1.12997293e-01  4.12405036e-01
   6.82579322e-01 -2.40227176e-01 -4.03299085e-01 -1.46244816e+00]
 [-1.07993121e-01 -5.78130229e-01 -2.60534432e-01  4.25447711e-01
  -3.09675104e-01 -1.05115239e+00  8.09609172e-01 -5.99011640e-01
   1.32392400e+00  3.76750790e-01  4.32374746e-01  3.31241182e-01
  -7.07358300e-01  8.71107667e-01  6.40302923e-01 -5.80139967e-01
  -9.73005200e-01  1.08982559e+00 -4.44014506e-01  1.62437026e-01]
 [ 1.04372217e+00 -9.71742600e-01  2.21321038e-01 -1.52941081e+00
   8.32702973e-01  8.96016179e-01  6.88742686e-01 -2.28811685e+00
   1.05503108e+00 -9.25040817e-01 -2.21412218e-01  3.74694327e-01
   1.02822107e+00  9.50047832e-01  1.03627399e-01  1.33274475e+00
  -1.51760432e

NameError: name 'W' is not defined

In [ ]:
def get_performance_averaging(smooth_std, fit_bin):
    smoothed, b = lr.smooth_spikes_unequal(s, sim_bin, fit_bin, smooth_std)
    sm_av = [smoothed[:int(n_trials/2)].mean(0)]
    A, pca = lr.fit_system(sm_av, d_latent, fit_bin)
    test_av = [smoothed[int(n_trials/2):].mean(0)]
    p,a = lr.one_step_diff_explained(test_av, A, pca, fit_bin)
    r2 = np.corrcoef(np.concatenate(p).flatten(), np.concatenate(a).flatten())[1,0]**2
    #r2 = r2_score(np.concatenate(p), np.concatenate(a))
    return r2,A,pca

In [ ]:
#iterative search for best smoothing parameters and fit bin parameters

r2 = []
smooth_stds = np.arange(0.01, 0.1, 0.01)
fit_bins = np.arange(0.01, 0.1, 0.01)
for smooth_std in smooth_stds:
    smooth_std = np.around(smooth_std, 2)
    r2.append(get_performance_averaging(smooth_std, fit_bin)[0])
    
smooth_std = smooth_stds[np.argmax(r2)]
r2 = []
fit_bins = np.arange(.01, .1, .01)
for fit_bin in fit_bins:
    fit_bin = np.around(fit_bin, 2)
    r2.append(get_performance_averaging(smooth_std, fit_bin)[0])
    
fit_bin = fit_bins[np.argmax(r2)]


In [ ]:
r2, A, pca = get_performance_averaging(smooth_std, fit_bin)
report_values(r2, A, pca)

# Linear-Gaussian models

## pylds implementation

In [ ]:
from pylds.models import DefaultLDS

model = DefaultLDS(100, d_latent)
_, b = lr.smooth_spikes(s, sim_bin, fit_bin, smooth_std)
for trial in b: model.add_data(trial)
ll = []
for i in range(100):
    model.resample_model()
    ll.append(model.log_likelihood())
    #print("step %d completed"%i)

for i in range(100):
    model.EM_step()
    ll.append(model.log_likelihood())
    #print("step %d completed"%i)
    
plt.plot(ll)

In [ ]:
from evaluate_all_datasets import evaluate_lds_difference

r2, p, a, x = evaluate_lds_difference(b, model)
print(r2)
print(np.corrcoef(np.concatenate(a).flatten(), np.concatenate(p).flatten()))

### Custom implementation
#### Square-root transform

In [ ]:
from glds import glds
from evaluate_all_datasets import evaluate_glds

fit_bin = 0.06
smooth_std = 0.06

model = glds(d_obs, d_latent)
smoothed, b = lr.smooth_spikes(s, sim_bin, fit_bin, smooth_std, sqtrans=True)
model.initialize(b)

for i in range(100):
    model.em_step()

In [ ]:
from evaluate_all_datasets import evaluate_glds
r2, p, a =  evaluate_glds(b, model)
print(r2)
print(np.corrcoef(np.concatenate(a).flatten(), np.concatenate(p).flatten()))

#### Untransformed

In [ ]:
from glds import glds
from evaluate_all_datasets import evaluate_glds

fit_bin = 0.06
smooth_std = 0.06

model = glds(d_obs, d_latent)
smoothed, b = lr.smooth_spikes(s, sim_bin, fit_bin, smooth_std)
model.initialize(b)

for i in range(100):
    model.em_step()

In [ ]:
from evaluate_all_datasets import evaluate_glds
r2, p, a =  evaluate_glds(b, model)
print(r2)
print(np.corrcoef(np.concatenate(a).flatten(), np.concatenate(p).flatten()))